In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
from sklearn.externals import joblib
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

%matplotlib inline

In [2]:
plt.rc('figure',dpi=100);

In [3]:
avisos = pd.read_csv("../../Data/fiuba_entrenamiento/gian/avisos.csv")
postulaciones = pd.read_csv("../../Data/fiuba_entrenamiento/gian/postulaciones.csv")
postulantes = pd.read_csv("../../Data/fiuba_entrenamiento/gian/postulantes.csv")

postulantes['sexo']=postulantes['sexo'].astype('category')
postulantes['Doctorado']=postulantes['Doctorado'].astype('category')
postulantes['Master']=postulantes['Master'].astype('category')
postulantes['Otro']=postulantes['Otro'].astype('category')
postulantes['Posgrado']=postulantes['Posgrado'].astype('category')
postulantes['Secundario']=postulantes['Secundario'].astype('category')
postulantes['Terciario/Técnico']=postulantes['Terciario/Técnico'].astype('category')
postulantes['Universitario']=postulantes['Universitario'].astype('category')

avisos['nombre_zona']=avisos['nombre_zona'].astype('category')
avisos['tipo_de_trabajo']=avisos['tipo_de_trabajo'].astype('category')
avisos['nivel_laboral']=avisos['nivel_laboral'].astype('category')
avisos['nombre_area']=avisos['nombre_area'].astype('category')

postulaciones['sepostulo']=1
postulaciones.drop(['fechapostulacion','dia','mes','semana','diadelasemana','hora'],axis=1,inplace=True)

In [4]:
vistas = pd.read_csv("../../Data/fiuba_entrenamiento/gian/vistas.csv")
vistas.head()

,idAviso,timestamp,idpostulante,dia,mes,semana,diadelasemana,hora
0,1111780242,2018-02-23 18:38:13.187,YjVJQ6Z,23,2,8,Viernes,18
1,1112263876,2018-02-23 18:38:14.296,BmVpYoR,23,2,8,Viernes,18
2,1112327963,2018-02-23 18:38:14.329,wVkBzZd,23,2,8,Viernes,18
3,1112318643,2018-02-23 18:38:17.921,OqmP9pv,23,2,8,Viernes,18
4,1111903673,2018-02-23 18:38:18.973,DrpbXDP,23,2,8,Viernes,18


In [5]:
vistas.drop(['timestamp','dia','mes','semana','diadelasemana','hora'],axis=1,inplace=True)
vistas['cantidad']=0
vistas=vistas.groupby(['idAviso','idpostulante'],as_index=False).agg({'cantidad':'count'})
vistas=vistas.rename(columns={'idAviso': 'idaviso'})
vistas.head()

,idaviso,idpostulante,cantidad
0,18,BolNL,2
1,48375,RwVdKR,1
2,169730,1KjXB,2
3,169730,2AKzxa,2
4,169730,6LJ64,1


In [6]:
postulaciones2= pd.read_csv("../../Data/fiuba_entrenamiento/gian/no-postulaciones/1.csv")

postulaciones=pd.merge(postulaciones,postulantes,on='idpostulante')
postulaciones2=pd.merge(postulaciones2,postulantes,on='idpostulante')

postulaciones=pd.merge(postulaciones,avisos,on='idaviso')
postulaciones2=pd.merge(postulaciones2,avisos,on='idaviso')

postulaciones2.head()

,idaviso,idpostulante,sepostulo,sexo,edad,Doctorado,Master,Otro,Posgrado,Secundario,...,secundario,viajar,empresa_importante,capacitacion,remuneracion_pretendida,graduados,estudiantes,conocimientos_especificos,buen_ambiente,discriminacion
0,1112257047,akjmXXq,0,FEM,28.0,-,-,-,-,Graduado,...,True,False,False,True,False,False,False,False,False,False
1,1112257047,MV6b6R9,0,FEM,30.0,-,-,-,-,-,...,True,False,False,True,False,False,False,False,False,False
2,1112257047,zvxv1Rm,0,FEM,22.0,-,-,-,-,Graduado,...,True,False,False,True,False,False,False,False,False,False
3,1112257047,8oqJzL,0,MASC,34.0,-,-,-,-,-,...,True,False,False,True,False,False,False,False,False,False
4,1112257047,ez533B,0,MASC,32.0,-,-,-,-,Graduado,...,True,False,False,True,False,False,False,False,False,False


In [7]:
postulaciones.head()

,idaviso,idpostulante,sepostulo,sexo,edad,Doctorado,Master,Otro,Posgrado,Secundario,...,secundario,viajar,empresa_importante,capacitacion,remuneracion_pretendida,graduados,estudiantes,conocimientos_especificos,buen_ambiente,discriminacion
0,1112257047,NM5M,1,FEM,47.0,-,-,-,-,Graduado,...,True,False,False,True,False,False,False,False,False,False
1,1112257047,1kJqGb,1,FEM,34.0,-,-,-,-,-,...,True,False,False,True,False,False,False,False,False,False
2,1112257047,eOE9Rr,1,FEM,39.0,-,-,-,-,Graduado,...,True,False,False,True,False,False,False,False,False,False
3,1112257047,Zrx8Xz,1,FEM,39.0,-,-,-,-,-,...,True,False,False,True,False,False,False,False,False,False
4,1112257047,ZrKNQY,1,FEM,32.0,-,-,-,-,Graduado,...,True,False,False,True,False,False,False,False,False,False


In [8]:
postulaciones=postulaciones.append(postulaciones2, ignore_index=True)
print(len(postulaciones))
postulaciones.drop_duplicates(['idaviso','idpostulante'],keep='first',inplace=True)
print(len(postulaciones))

12377268
12332167


In [9]:
postulaciones=pd.merge(postulaciones,vistas,on=['idaviso','idpostulante'],how='left')
postulaciones.sample(20)

,idaviso,idpostulante,sepostulo,sexo,edad,Doctorado,Master,Otro,Posgrado,Secundario,...,viajar,empresa_importante,capacitacion,remuneracion_pretendida,graduados,estudiantes,conocimientos_especificos,buen_ambiente,discriminacion,cantidad
2474748,1112250302,6VoK33,1,FEM,30.0,-,-,-,-,Graduado,...,False,False,False,False,False,True,False,True,False,NaN
11741981,1112235184,EzlYK1z,0,MASC,20.0,-,-,-,-,Graduado,...,True,False,False,False,False,False,False,False,False,NaN
821488,1112431304,MV68vex,1,FEM,21.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,False,1.0
5731589,1112299637,0zdAM91,1,MASC,24.0,-,-,-,-,Graduado,...,False,False,False,False,False,True,False,False,False,NaN
6345754,1112380204,KBrDRVm,0,MASC,23.0,-,-,-,-,Graduado,...,False,True,False,False,False,False,False,False,False,NaN
2326207,1112368915,vV3Jk2X,1,FEM,28.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,False,NaN
5526830,1112258964,KBr6jem,1,MASC,43.0,-,-,-,-,Graduado,...,False,False,False,True,False,False,False,False,False,NaN
2928292,1112257808,3NLAeDX,1,FEM,53.0,-,-,-,-,-,...,False,False,False,False,False,True,True,False,True,4.0
9499507,1112467175,zvam3bK,0,FEM,32.0,-,-,-,-,-,...,False,False,True,False,False,True,False,True,False,NaN
3849794,1112256839,96XrbGv,1,MASC,27.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,True,False,NaN


In [10]:
postulaciones.cantidad=postulaciones.cantidad.fillna(0)
postulaciones.sample(20)

,idaviso,idpostulante,sepostulo,sexo,edad,Doctorado,Master,Otro,Posgrado,Secundario,...,viajar,empresa_importante,capacitacion,remuneracion_pretendida,graduados,estudiantes,conocimientos_especificos,buen_ambiente,discriminacion,cantidad
1526286,1111662226,ow2QGPe,1,FEM,23.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,False,0.0
3513082,1112368896,aPZEVE,1,FEM,32.0,-,En Curso,-,En Curso,Graduado,...,False,True,False,False,False,False,False,False,False,0.0
11007300,1112015658,xkdANmb,0,MASC,21.0,-,-,-,-,-,...,False,True,False,False,False,False,True,False,True,0.0
5315791,1112439046,JBre5Xb,1,FEM,44.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,False,4.0
979593,1112204682,xkdq8d0,1,MASC,18.0,-,-,-,-,-,...,False,False,True,False,False,False,False,True,False,0.0
9815923,1112309787,96Za3kP,0,FEM,28.0,-,-,-,-,Graduado,...,False,True,False,False,False,False,True,False,True,0.0
3033232,1112461642,mzjG2M6,1,FEM,34.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,False,0.0
1633716,1112403264,EzEJ1o4,1,FEM,24.0,-,-,-,-,Graduado,...,False,False,True,False,False,False,False,False,False,0.0
4787776,1112348497,ow2LP44,1,MASC,20.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,False,0.0
2422026,1112273698,8bmVbW,1,MASC,31.0,-,-,-,-,-,...,False,False,False,False,False,False,False,False,False,0.0


In [11]:
zonas=avisos['nombre_zona'].value_counts().index
postulaciones['nombre_zona']=postulaciones['nombre_zona'].replace(zonas, range(len(zonas)))

tipos_de_trabajo=avisos['tipo_de_trabajo'].value_counts().index
postulaciones['tipo_de_trabajo']=postulaciones['tipo_de_trabajo'].replace(tipos_de_trabajo, range(len(tipos_de_trabajo)))

niveles=avisos['nivel_laboral'].value_counts().index
postulaciones['nivel_laboral']=postulaciones['nivel_laboral'].replace(niveles, range(len(niveles)))

areas=avisos['nombre_area'].value_counts().index
postulaciones['nombre_area']=postulaciones['nombre_area'].replace(areas, range(len(areas)))

doct=postulantes['Doctorado'].value_counts().index
postulaciones['Doctorado']=postulaciones['Doctorado'].replace(doct, range(len(doct)))

master=postulantes['Master'].value_counts().index
postulaciones['Master']=postulaciones['Master'].replace(master, range(len(master)))

otro=postulantes['Otro'].value_counts().index
postulaciones['Otro']=postulaciones['Otro'].replace(otro, range(len(otro)))

posgrado=postulantes['Posgrado'].value_counts().index
postulaciones['Posgrado']=postulaciones['Posgrado'].replace(posgrado, range(len(posgrado)))

sec=postulantes['Secundario'].value_counts().index
postulaciones['Secundario']=postulaciones['Secundario'].replace(sec, range(len(sec)))

terc=postulantes['Terciario/Técnico'].value_counts().index
postulaciones['Terciario/Técnico']=postulaciones['Terciario/Técnico'].replace(terc, range(len(terc)))

univ=postulantes['Universitario'].value_counts().index
postulaciones['Universitario']=postulaciones['Universitario'].replace(univ, range(len(univ)))

sexo=postulantes['sexo'].value_counts().index
postulaciones['sexo']=postulaciones['sexo'].replace(sexo, range(len(sexo)))
postulaciones.dtypes

idaviso                        int64
idpostulante                  object
sepostulo                      int64
sexo                           int64
edad                         float64
Doctorado                      int64
Master                         int64
Otro                           int64
Posgrado                       int64
Secundario                     int64
Terciario/Técnico              int64
Universitario                  int64
titulo                        object
nombre_zona                    int64
tipo_de_trabajo                int64
nivel_laboral                  int64
nombre_area                    int64
denominacion_empresa          object
titulo_uppercase                bool
pide_hombre                     bool
pide_mujer                      bool
ingles                          bool
experiencia                     bool
paquete_office                  bool
liderazgo                       bool
secundario                      bool
viajar                          bool
e

In [12]:
train, test=train_test_split(postulaciones,test_size=0.10)
print("Train: ",len(train),"Test: ",len(test))
features=list(postulaciones.columns)
features.remove('idaviso')
features.remove('idpostulante')
features.remove('sepostulo')
features.remove('titulo')
features.remove('denominacion_empresa')

x_train=train[features]
y_train=train['sepostulo']

x_test=test[features]
y_test=test['sepostulo']

Train:  11098950 Test:  1233217


In [13]:
c=XGBClassifier(objective='binary:logistic',max_depth=13,learning_rate=0.1,n_estimators=500,silent=False,n_jobs=-1)
#c = joblib.load("../../Data/fiuba_entrenamiento/gian/modelos/....pkl"
dt=c.fit(x_train,y_train)

In [14]:
score=c.score(x_test,y_test)*100
print(score)

74.88576625200594


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [15]:
joblib.dump(c, "../../Data/fiuba_entrenamiento/gian/modelos/XG1.pkl")

['../../Data/fiuba_entrenamiento/gian/modelos/XG1.pkl']

In [16]:
prediccion = pd.read_csv("../../Data/fiuba_entrenamiento/test_final_100k.csv")
print(len(prediccion))
prediccion=pd.merge(prediccion,postulantes,on='idpostulante',how='inner')
print(len(prediccion))
prediccion=pd.merge(prediccion,avisos,on='idaviso',how='inner')
print(len(prediccion))
prediccion=pd.merge(prediccion,vistas,on=['idaviso','idpostulante'],how='left')
prediccion.cantidad=prediccion.cantidad.fillna(0)
prediccion.sample(10)

100000
100000
100000


,id,idaviso,idpostulante,sexo,edad,Doctorado,Master,Otro,Posgrado,Secundario,...,viajar,empresa_importante,capacitacion,remuneracion_pretendida,graduados,estudiantes,conocimientos_especificos,buen_ambiente,discriminacion,cantidad
29007,69963,1112461641,Oqb9lj8,MASC,20.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,True,False,2.0
92803,91896,1112473306,E8x0G0,FEM,34.0,-,-,-,-,-,...,False,False,False,False,True,True,False,False,False,0.0
74429,20525,1112257808,5MAJDq,FEM,34.0,-,-,-,-,Graduado,...,False,False,False,False,False,True,True,False,True,0.0
60470,43526,1112425147,YAQ0XJ,FEM,34.0,-,-,Graduado,-,Graduado,...,False,False,False,False,False,False,False,False,False,0.0
79427,26964,1112346768,a0w4ND,FEM,34.0,-,-,-,-,Graduado,...,False,False,False,False,True,False,False,False,False,0.0
47512,8376,1111775363,ZDr1ZlY,FEM,20.0,-,-,-,-,Graduado,...,False,True,True,True,False,False,False,False,False,2.0
63304,81502,1112466083,52Y8Zk,MASC,62.0,-,Graduado,-,-,Graduado,...,False,False,False,False,False,False,False,False,False,0.0
89819,90822,1112468924,NrpWY5,MASC,35.0,-,-,-,-,Graduado,...,False,False,True,False,True,True,False,False,False,0.0
21406,83185,1112466360,qev3QR9,MASC,38.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,False,1.0
45895,1962,1111170254,NrKwZl,FEM,38.0,-,-,Graduado,Graduado,Graduado,...,False,False,False,False,False,False,False,False,False,0.0


In [17]:
zonas=avisos['nombre_zona'].value_counts().index
prediccion['nombre_zona']=prediccion['nombre_zona'].replace(zonas, range(len(zonas)))

tipos_de_trabajo=avisos['tipo_de_trabajo'].value_counts().index
prediccion['tipo_de_trabajo']=prediccion['tipo_de_trabajo'].replace(tipos_de_trabajo, range(len(tipos_de_trabajo)))

niveles=avisos['nivel_laboral'].value_counts().index
prediccion['nivel_laboral']=prediccion['nivel_laboral'].replace(niveles, range(len(niveles)))

areas=avisos['nombre_area'].value_counts().index
prediccion['nombre_area']=prediccion['nombre_area'].replace(areas, range(len(areas)))

doct=postulantes['Doctorado'].value_counts().index
prediccion['Doctorado']=prediccion['Doctorado'].replace(doct, range(len(doct)))

master=postulantes['Master'].value_counts().index
prediccion['Master']=prediccion['Master'].replace(master, range(len(master)))

otro=postulantes['Otro'].value_counts().index
prediccion['Otro']=prediccion['Otro'].replace(otro, range(len(otro)))

posgrado=postulantes['Posgrado'].value_counts().index
prediccion['Posgrado']=prediccion['Posgrado'].replace(posgrado, range(len(posgrado)))

sec=postulantes['Secundario'].value_counts().index
prediccion['Secundario']=prediccion['Secundario'].replace(sec, range(len(sec)))

terc=postulantes['Terciario/Técnico'].value_counts().index
prediccion['Terciario/Técnico']=prediccion['Terciario/Técnico'].replace(terc, range(len(terc)))

univ=postulantes['Universitario'].value_counts().index
prediccion['Universitario']=prediccion['Universitario'].replace(univ, range(len(univ)))

sexo=postulantes['sexo'].value_counts().index
prediccion['sexo']=prediccion['sexo'].replace(sexo, range(len(sexo)))
prediccion.dtypes

id                             int64
idaviso                        int64
idpostulante                  object
sexo                           int64
edad                         float64
Doctorado                      int64
Master                         int64
Otro                           int64
Posgrado                       int64
Secundario                     int64
Terciario/Técnico              int64
Universitario                  int64
titulo                        object
nombre_zona                    int64
tipo_de_trabajo                int64
nivel_laboral                  int64
nombre_area                    int64
denominacion_empresa          object
titulo_uppercase                bool
pide_hombre                     bool
pide_mujer                      bool
ingles                          bool
experiencia                     bool
paquete_office                  bool
liderazgo                       bool
secundario                      bool
viajar                          bool
e

In [18]:
x_final=prediccion[features]
y_final=c.predict(x_final)
y_final

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [19]:
prediccion['sepostulo']=y_final

In [20]:
prediccion.head()

,id,idaviso,idpostulante,sexo,edad,Doctorado,Master,Otro,Posgrado,Secundario,...,empresa_importante,capacitacion,remuneracion_pretendida,graduados,estudiantes,conocimientos_especificos,buen_ambiente,discriminacion,cantidad,sepostulo
0,0,739260,6M9ZQR,0,42.0,1,0,0,0,0,...,False,False,False,False,False,False,False,False,0.0,0
1,1,739260,6v1xdL,1,30.0,0,0,1,0,1,...,False,False,False,False,False,False,False,False,0.0,0
2,2,739260,ezRKm9,0,36.0,0,0,0,0,0,...,False,False,False,False,False,False,False,False,0.0,0
3,3,758580,1Q35ej,1,68.0,0,0,0,1,0,...,False,False,False,False,False,False,False,False,0.0,0
4,4,758580,EAN4J6,0,32.0,0,0,0,0,1,...,False,False,False,False,False,False,False,False,0.0,0


In [21]:
prediccion.drop(features,axis=1,inplace=True)
prediccion.drop(['idaviso','idpostulante','titulo','denominacion_empresa'],axis=1,inplace=True)
prediccion.head()

,id,sepostulo
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0


In [22]:
prediccion.to_csv("../../Data/fiuba_entrenamiento/gian/predicciones/prediccion8.csv",index=False)

In [23]:
prediccion.sepostulo.mean()

0.53751

In [24]:
prediccion = pd.read_csv("../../Data/fiuba_entrenamiento/test_final_100k.csv")
print(len(prediccion))
prediccion=pd.merge(prediccion,postulantes,on='idpostulante',how='inner')
print(len(prediccion))
prediccion=pd.merge(prediccion,avisos,on='idaviso',how='inner')
print(len(prediccion))
prediccion=pd.merge(prediccion,vistas,on=['idaviso','idpostulante'],how='left')
prediccion.cantidad=prediccion.cantidad.fillna(0)
prediccion.sample(10)

100000
100000
100000


,id,idaviso,idpostulante,sexo,edad,Doctorado,Master,Otro,Posgrado,Secundario,...,viajar,empresa_importante,capacitacion,remuneracion_pretendida,graduados,estudiantes,conocimientos_especificos,buen_ambiente,discriminacion,cantidad
56759,85834,1112466690,eO3QX4,MASC,34.0,-,En Curso,-,-,Graduado,...,False,False,True,False,False,False,False,False,False,1.0
17712,73122,1112463317,4rNzLje,FEM,24.0,-,-,-,-,Graduado,...,False,False,False,True,False,False,False,True,False,0.0
72686,49147,1112437476,A3DwreL,FEM,28.0,-,-,-,-,Graduado,...,False,True,False,False,False,False,False,True,False,4.0
5363,33921,1112405655,1Qq48VN,FEM,55.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,False,2.0
34455,34368,1112408569,YQDrD,NO_DECLARA,200.0,-,-,-,-,-,...,False,False,False,False,False,False,True,False,True,0.0
28787,20936,1112264738,RzrB5p9,FEM,42.0,-,-,-,-,Graduado,...,False,False,False,False,False,True,False,True,False,0.0
1282,80807,1112465244,lDLQ6W8,FEM,33.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,False,1.0
20526,83465,1112466383,LN8528j,MASC,23.0,-,-,-,-,Graduado,...,False,True,False,False,False,False,False,False,False,0.0
84941,33749,1112404239,ep2qE9,MASC,34.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,False,0.0
41026,29562,1112372247,dYoJeW0,MASC,28.0,-,-,-,-,Graduado,...,False,True,False,False,False,False,False,False,False,1.0


In [25]:
zonas=avisos['nombre_zona'].value_counts().index
prediccion['nombre_zona']=prediccion['nombre_zona'].replace(zonas, range(len(zonas)))

tipos_de_trabajo=avisos['tipo_de_trabajo'].value_counts().index
prediccion['tipo_de_trabajo']=prediccion['tipo_de_trabajo'].replace(tipos_de_trabajo, range(len(tipos_de_trabajo)))

niveles=avisos['nivel_laboral'].value_counts().index
prediccion['nivel_laboral']=prediccion['nivel_laboral'].replace(niveles, range(len(niveles)))

areas=avisos['nombre_area'].value_counts().index
prediccion['nombre_area']=prediccion['nombre_area'].replace(areas, range(len(areas)))

doct=postulantes['Doctorado'].value_counts().index
prediccion['Doctorado']=prediccion['Doctorado'].replace(doct, range(len(doct)))

master=postulantes['Master'].value_counts().index
prediccion['Master']=prediccion['Master'].replace(master, range(len(master)))

otro=postulantes['Otro'].value_counts().index
prediccion['Otro']=prediccion['Otro'].replace(otro, range(len(otro)))

posgrado=postulantes['Posgrado'].value_counts().index
prediccion['Posgrado']=prediccion['Posgrado'].replace(posgrado, range(len(posgrado)))

sec=postulantes['Secundario'].value_counts().index
prediccion['Secundario']=prediccion['Secundario'].replace(sec, range(len(sec)))

terc=postulantes['Terciario/Técnico'].value_counts().index
prediccion['Terciario/Técnico']=prediccion['Terciario/Técnico'].replace(terc, range(len(terc)))

univ=postulantes['Universitario'].value_counts().index
prediccion['Universitario']=prediccion['Universitario'].replace(univ, range(len(univ)))

sexo=postulantes['sexo'].value_counts().index
prediccion['sexo']=prediccion['sexo'].replace(sexo, range(len(sexo)))
prediccion.dtypes

id                             int64
idaviso                        int64
idpostulante                  object
sexo                           int64
edad                         float64
Doctorado                      int64
Master                         int64
Otro                           int64
Posgrado                       int64
Secundario                     int64
Terciario/Técnico              int64
Universitario                  int64
titulo                        object
nombre_zona                    int64
tipo_de_trabajo                int64
nivel_laboral                  int64
nombre_area                    int64
denominacion_empresa          object
titulo_uppercase                bool
pide_hombre                     bool
pide_mujer                      bool
ingles                          bool
experiencia                     bool
paquete_office                  bool
liderazgo                       bool
secundario                      bool
viajar                          bool
e

In [26]:
x_final=prediccion[features]
y_final=c.predict_proba(x_final)
y_final

array([[0.93359905, 0.06640092],
       [0.58801836, 0.41198164],
       [0.6202413 , 0.3797587 ],
       ...,
       [0.92469245, 0.07530753],
       [0.68566567, 0.31433433],
       [0.9813561 , 0.01864392]], dtype=float32)

In [27]:
y_final2=[]
print(c.classes_)
print(len(y_final))
for elem in y_final:
    y_final2.append(elem[1])
print(len(y_final2))

[0 1]
100000
100000


In [28]:
prediccion['sepostulo']=y_final2

In [29]:
prediccion.head()

,id,idaviso,idpostulante,sexo,edad,Doctorado,Master,Otro,Posgrado,Secundario,...,empresa_importante,capacitacion,remuneracion_pretendida,graduados,estudiantes,conocimientos_especificos,buen_ambiente,discriminacion,cantidad,sepostulo
0,0,739260,6M9ZQR,0,42.0,1,0,0,0,0,...,False,False,False,False,False,False,False,False,0.0,0.066401
1,1,739260,6v1xdL,1,30.0,0,0,1,0,1,...,False,False,False,False,False,False,False,False,0.0,0.411982
2,2,739260,ezRKm9,0,36.0,0,0,0,0,0,...,False,False,False,False,False,False,False,False,0.0,0.379759
3,3,758580,1Q35ej,1,68.0,0,0,0,1,0,...,False,False,False,False,False,False,False,False,0.0,0.106857
4,4,758580,EAN4J6,0,32.0,0,0,0,0,1,...,False,False,False,False,False,False,False,False,0.0,0.392633


In [30]:
prediccion.drop(features,axis=1,inplace=True)
prediccion.drop(['idaviso','idpostulante','titulo','denominacion_empresa'],axis=1,inplace=True)
prediccion.head()

,id,sepostulo
0,0,0.066401
1,1,0.411982
2,2,0.379759
3,3,0.106857
4,4,0.392633


In [31]:
prediccion.to_csv("../../Data/fiuba_entrenamiento/gian/predicciones/prediccion9.csv",index=False)

In [32]:
prediccion.sepostulo.mean()

0.5924806174768673